In [1]:
# Cody Baker, June 15 2020
#
# This is code used to understand and track the inner workings of a refactor for convert_yuta.py

import os
from dateutil.parser import parse as dateparse
from datetime import datetime
import pandas as pd
import numpy as np

from pynwb import NWBFile, NWBHDF5IO, TimeSeries
from pynwb.behavior import SpatialSeries, Position
from pynwb.file import Subject, TimeIntervals
from pynwb.misc import DecompositionSeries

# had to find and add the to_nwb package to the folder; will try making a path pointer later
import to_nwb.neuroscope as ns

session_path='D:\BuzsakiData\SenzaiY\YutaMouse41\YutaMouse41-150903'
subject_xls=None
include_spike_waveforms=True
stub=True
cache_spec=True


subject_path, session_id = os.path.split(session_path)
fpath_base = os.path.split(subject_path)[0]
identifier = session_id
mouse_number = session_id[9:11] # somewhat manual, what if single or triple digits?

In [2]:
print(subject_path)
print(session_id)
print(fpath_base)
print(identifier)
print(mouse_number)

D:\BuzsakiData\SenzaiY\YutaMouse41
YutaMouse41-150903
D:\BuzsakiData\SenzaiY
YutaMouse41-150903
41


In [3]:
if '-' in session_id:
    subject_id, date_text = session_id.split('-')
    b = False
else:
    subject_id, date_text = session_id.split('b') # not sure what condition this is for outside the Yuta datasets
    b = True

In [4]:
print(subject_id)
print(date_text)
print(b)

YutaMouse41
150903
False


In [5]:
if subject_xls is None:
    subject_xls = os.path.join(subject_path, 'YM' + mouse_number + ' exp_sheet.xlsx') # Cody: this first condition I understand
else:
    if not subject_xls[-4:] == 'xlsx':
        subject_xls = os.path.join(subject_xls, 'YM' + mouse_number + ' exp_sheet.xlsx') # Cody: this one seems odd
        
# Cody: also, the data I recently retrieved from the lab has no .xlsx files

In [6]:
print(subject_xls)

D:\BuzsakiData\SenzaiY\YutaMouse41\YM41 exp_sheet.xlsx


In [7]:
session_start_time = dateparse(date_text, yearfirst=True)
print(session_start_time)

2015-09-03 00:00:00


In [8]:
df = pd.read_excel(subject_xls)

In [9]:
subject_data = {}
for key in ['genotype', 'DOB', 'implantation', 'Probe', 'Surgery', 'virus injection', 'mouseID']:
    names = df.iloc[:, 0]
    if key in names.values:
        subject_data[key] = df.iloc[np.argmax(names == key), 1]
        
if isinstance(subject_data['DOB'], datetime):
    age = session_start_time - subject_data['DOB']
else:
    age = None

subject = Subject(subject_id=subject_id, age=str(age),
                  genotype=subject_data['genotype'],
                  species='mouse') # I thought species was supposed to be the latin name?

In [18]:
nwbfile = NWBFile(session_description='mouse in open exploration and theta maze',
                      identifier=identifier,
                      session_start_time=session_start_time.astimezone(),
                      file_create_date=datetime.now().astimezone(),
                      experimenter='Yuta Senzai',
                      session_id=session_id,
                      institution='NYU',
                      lab='Buzsaki',
                      subject=subject,
                      related_publications='DOI:10.1016/j.neuron.2016.12.011')

In [19]:
nwbfile

root pynwb.file.NWBFile at 0x2896137111496
Fields:
  experimenter: ['Yuta Senzai']
  file_create_date: [datetime.datetime(2020, 6, 15, 13, 55, 5, 172076, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'US Eastern Daylight Time'))]
  identifier: YutaMouse41-150903
  institution: NYU
  lab: Buzsaki
  related_publications: ['DOI:10.1016/j.neuron.2016.12.011']
  session_description: mouse in open exploration and theta maze
  session_id: YutaMouse41-150903
  session_start_time: 2015-09-03 00:00:00-04:00
  subject: subject pynwb.file.Subject at 0x2896132298120
Fields:
  age: 346 days 00:00:00
  genotype: POMC-Cre::Arch
  species: mouse
  subject_id: YutaMouse41

  timestamps_reference_time: 2015-09-03 00:00:00-04:00

In [11]:
print('reading and writing raw position data...', end='', flush=True)
ns.add_position_data(nwbfile, session_path)

reading and writing raw position data...warning: time may not be aligned


In [12]:
nwbfile.acquisition['position_sensor0'].data[()]

array([[   nan,    nan],
       [   nan,    nan],
       [   nan,    nan],
       ...,
       [362.  , 171.  ],
       [362.  , 171.13],
       [362.  , 171.94]])

In [13]:
shank_channels = ns.get_shank_channels(session_path)[:8] # Cody: why [:8]?
all_shank_channels = np.concatenate(shank_channels)

In [14]:
shank_channels

[[33, 38, 32, 39, 35, 36, 34, 37],
 [41, 46, 40, 47, 43, 44, 42, 45],
 [16, 25, 17, 24, 18, 22, 20, 21],
 [26, 19, 28, 23, 29, 27, 30, 31],
 [13, 4, 9, 2, 5, 3, 1, 0],
 [7, 14, 6, 15, 8, 12, 11, 10],
 [48, 55, 49, 54, 50, 53, 51, 52],
 [56, 63, 57, 62, 58, 61, 59, 60]]

In [15]:
ns.get_shank_channels(session_path)

[[33, 38, 32, 39, 35, 36, 34, 37],
 [41, 46, 40, 47, 43, 44, 42, 45],
 [16, 25, 17, 24, 18, 22, 20, 21],
 [26, 19, 28, 23, 29, 27, 30, 31],
 [13, 4, 9, 2, 5, 3, 1, 0],
 [7, 14, 6, 15, 8, 12, 11, 10],
 [48, 55, 49, 54, 50, 53, 51, 52],
 [56, 63, 57, 62, 58, 61, 59, 60]]

In [16]:
print('setting up electrodes...', end='', flush=True)
hilus_csv_path = os.path.join(fpath_base, 'early_session_hilus_chans.csv')
lfp_channel = get_reference_elec(subject_xls, hilus_csv_path, session_start_time, session_id, b=b)
print(lfp_channel)
custom_column = [{'name': 'theta_reference',
                  'description': 'this electrode was used to calculate LFP canonical bands',
                  'data': all_shank_channels == lfp_channel}]
ns.write_electrode_table(nwbfile, session_path, custom_columns=custom_column, max_shanks=max_shanks)

print('reading LFPs...', end='', flush=True)
lfp_fs, all_channels_data = ns.read_lfp(session_path, stub=stub)

lfp_data = all_channels_data[:, all_shank_channels]
print('writing LFPs...', flush=True)
# lfp_data[:int(len(lfp_data)/4)]
lfp_ts = ns.write_lfp(nwbfile, lfp_data, lfp_fs, name='lfp',
                      description='lfp signal for all shank electrodes')

for name, channel in special_electrode_dict.items():
    ts = TimeSeries(name=name, description='environmental electrode recorded inline with neural data',
                    data=all_channels_data[:, channel], rate=lfp_fs, unit='V', conversion=np.nan, resolution=np.nan)
    nwbfile.add_acquisition(ts)

setting up electrodes...

NameError: name 'get_reference_elec' is not defined